<a href="https://colab.research.google.com/github/abhi247ai/tensorflow_sessions/blob/main/Text_Generation_NLP_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

**Loading Text File**

Method : 1 from googe databse as shown and used (shakespeare.txt)

Method2 : 2 using our own file.txt, by executiong the following code

from.google.colab import files


path_to_file = list(files.upload().keys())[0]

In [59]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

** Read the file **

In [60]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print('Length of text: {} characters'. format(len(text)))

print(text[:250])

Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



**Text-to-Integer: Encoding**

In [61]:
vocab = sorted(set(text))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [62]:
print('Text:', text[:13])
print('Encoded:', text_to_int(text[:13]))


Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [63]:
# function for converting int to text

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [64]:
#creating sequence of text data

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [65]:
# now make the batches of the sequences

sequences = char_dataset.batch(seq_length+1, drop_remainder = True)

Now, sequnce length of 101 is splitted in to input and output

In [66]:
def split_input_target(chunk):    #chunk is hello
  input_text = chunk[:-1] #hell
  target_text = chunk[1:] #ello
  return input_text, target_text   #hell, #ello

dataset = sequences.map(split_input_target)  #it applies every entry this function

In [67]:
for x,y in dataset.take(2):
  print("\n\nExample\n")
  print("INPUT")
  print(int_to_text(x))
  print('\nOUTPUT')
  print(int_to_text(y))



Example

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


Example

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [68]:
# MAKE TRAINING BATCHES

BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  #vovan is number of unique character
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000   #shuffle this size of sequence intead of entire.

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)



**MODEL**

In [69]:
# will make a function

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                                                         batch_input_shape = [batch_size, None]),
                               tf.keras.layers.LSTM(rnn_units,
                                                    return_sequences = True,
                                                    stateful = True,
                                                    recurrent_initializer = 'glorot_uniform'),
                               tf.keras.layers.Dense(vocab_size)
  ])

  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


**LOSS FUNCTION**

In [70]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)

(64, 100, 65)


In [71]:
# see prediction in 64 array of array 
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-3.62738974e-05 -3.26399948e-03 -2.11016973e-03 ... -2.81044049e-04
    1.82781753e-03 -1.60905148e-03]
  [ 4.77091968e-03 -1.04946271e-02  8.58562998e-03 ...  5.99420280e-04
    8.38554557e-03  3.38657899e-03]
  [ 7.02647376e-04 -3.73084075e-03  1.17671993e-02 ... -5.91073302e-04
    1.02513414e-02  3.70128616e-03]
  ...
  [ 7.28499424e-03 -7.00466847e-03  1.28159509e-03 ...  1.25183375e-03
   -2.43848586e-03 -3.87968216e-03]
  [ 5.95462834e-03 -1.06190527e-02  3.08870873e-03 ...  1.56158453e-03
   -8.22332129e-03 -5.36829326e-03]
  [ 4.78641037e-03 -1.37426145e-02  4.68015485e-03 ...  1.69402757e-03
   -1.21643087e-02 -7.03235902e-03]]

 [[-1.30369712e-03  6.83549326e-03 -6.20381848e-04 ... -1.16400234e-03
    2.00667419e-05 -2.70093884e-03]
  [ 2.05996563e-03  8.59704707e-03 -4.37307026e-04 ... -2.76869582e-03
   -3.41496989e-03  2.99177482e-03]
  [ 2.77748005e-03  8.27050488e-03 -4.80622344e-04 ... -4.97276150e-03
   -3.96913104e-03  1.03526819e-03]
  ...
  [ 5.558

In [72]:
# see one prediction for example
pred = example_batch_predictions[0]
print(len(pred))
print(pred)


100
tf.Tensor(
[[-3.6273897e-05 -3.2639995e-03 -2.1101697e-03 ... -2.8104405e-04
   1.8278175e-03 -1.6090515e-03]
 [ 4.7709197e-03 -1.0494627e-02  8.5856300e-03 ...  5.9942028e-04
   8.3855456e-03  3.3865790e-03]
 [ 7.0264738e-04 -3.7308407e-03  1.1767199e-02 ... -5.9107330e-04
   1.0251341e-02  3.7012862e-03]
 ...
 [ 7.2849942e-03 -7.0046685e-03  1.2815951e-03 ...  1.2518337e-03
  -2.4384859e-03 -3.8796822e-03]
 [ 5.9546283e-03 -1.0619053e-02  3.0887087e-03 ...  1.5615845e-03
  -8.2233213e-03 -5.3682933e-03]
 [ 4.7864104e-03 -1.3742615e-02  4.6801548e-03 ...  1.6940276e-03
  -1.2164309e-02 -7.0323590e-03]], shape=(100, 65), dtype=float32)


In [73]:
# see prediction at first time step
#probability of each character occuring next
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[-3.6273897e-05 -3.2639995e-03 -2.1101697e-03  5.2232072e-03
 -1.8209992e-03  2.3395782e-03  4.2266655e-03  1.1061723e-03
 -2.4870788e-03  5.1675751e-03 -1.5849890e-03  1.5781400e-03
  5.8703860e-03  6.1892741e-04 -5.9137377e-04  5.3535524e-04
  3.9452673e-03  1.8023144e-03 -2.1675143e-03 -1.5955428e-03
 -6.6801966e-03  3.5678402e-03 -2.7283528e-03 -1.4853654e-03
 -4.7636968e-03  1.0023562e-04  3.9076163e-03 -9.0568181e-04
  7.1107422e-04  5.5392403e-03  4.2444989e-04 -7.9077622e-03
  3.1778754e-03  1.8600707e-03  7.1335485e-04  1.4725857e-03
  4.7493684e-03 -1.2919090e-03 -3.0939313e-04 -2.2662033e-03
  1.7559600e-04 -1.2480476e-03  2.7443704e-03  1.0931890e-03
  2.0495008e-03 -3.0489557e-03 -2.3698714e-03 -4.1487329e-03
 -7.9927396e-04  4.8768311e-04  2.9980141e-04  3.5392428e-03
  6.6022137e-03  6.1414484e-03 -5.4399497e-03  2.9606712e-03
 -9.7407268e-05 -9.4529637e-04  1.9215900e-03  2.1577836e-03
 -3.0165285e-04 -1.5285509e-03 -2.8104405e-04  1.8278175e-03
 -1.609051

In [74]:
#to see the predicted characters

sample_indices = tf.random.categorical(pred, num_samples=1)

sample_indices = np.reshape(sample_indices, (1,-1))[0]
predicted_chars = int_to_text(sample_indices)

predicted_chars

"cPy ?&a:U$Vv-\n.EB-\nQcn&,OI,z;vYGoj!awdFt'T\nzteuZqp!VG?MBaVMF;OnGeqOmTESfQw!fXX 3wPTc'SFoKIlr!BjYUp$S"

In [75]:
# now create a loss function

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


**MODEL COMPILE**

In [76]:
model.compile(optimizer = 'adam', loss = loss)

**Creating Checkpoints**

save checkpoints as model trains, we cam load model from checkpoint and continue training it.

In [77]:
# directory to save checkpoints

checkpoint_dir = './training_checkpoints'

#name of checkpoint file

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

**Training**

In [78]:
history = model.fit(data, epochs = 1, callbacks = [checkpoint_callback])

172/172 [==============================] - 33s 174ms/step - loss: 2.5806


**LOADING MODEL**

rebuild for batch size 1, so we can feed one sentence to test the prediction.

In [79]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size = 1)   #in batch size 1

In [88]:
#to find the checkpoints 
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))


In [90]:
#we can also load specified checkpoint

checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint('./content/training/checkpoints/ckpt_'+ str(checkpoint_num))
model.build(tf.TensorShape([1, None]))

SyntaxError: ignored

In [81]:
#function for any length of text

def generate_text(model, start_string):

  num_generate = 800     #num of char to generate

  #convert string to num(vectorization)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  #empty list to store string
  text_generated = []

  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    #remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions/temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    #pass predicted output as next input to the model
    #along with previous hidden state

    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [82]:
inp = input("Type a startig string:")
print(generate_text(model, inp))

Type a startig string:romeo
romeou, thy and meney
You forkaun ann. I'dading hard bagt ince ay hilp?

PARTES:
The withtur apsod, dor, benet; Buenthare hest akn ve ang Youed,
Be vand ane you puru, to meme them
your murpeand.

ARLIS HASNILK:
I pade read astsimon ;
Chaign beived,
Sele steirird-st?
Thet so aio duth Slexand aserpprem's!
Benouts whine of ugpricand mird ruby yium, thare's prear.

AREMIO:
Ss mard ne soust fores ale and?

MALTERRES:
Cithoud, icatist thy pamiis bater like.

COMETCARCEO:
Banour I than shalv at wille noon.

LERCERTaOing atlond of unwiot nis.

FERENIS:
Ay wo the fresting, mensid winte yoo whall d the sadovt,ssencloutter afly ryoit
Sreafur Camonts--urntly wers meaded;
Herer miecy the gamknous yot rrey
But be not.
BAO fopt:
Aft'ss eperty:
IJk hid shey sher fhip of ast to bidit mante, sen hey ant thou nal
